#### Import necessary packages


In [20]:
# Standard library imports
import math
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

# Third-party imports
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from einops import einsum, rearrange
from torchvision.datasets import ImageNet
from torchvision.models import AlexNet_Weights

#### Load the weights and biases of AlexNet


In [114]:
weights_and_biases = AlexNet_Weights.DEFAULT.get_state_dict()
print(weights_and_biases.keys())

odict_keys(['features.0.weight', 'features.0.bias', 'features.3.weight', 'features.3.bias', 'features.6.weight', 'features.6.bias', 'features.8.weight', 'features.8.bias', 'features.10.weight', 'features.10.bias', 'classifier.1.weight', 'classifier.1.bias', 'classifier.4.weight', 'classifier.4.bias', 'classifier.6.weight', 'classifier.6.bias'])


#### Load the data


In [85]:
def default_collate(batch):
    """
    A collation function that simply returns the batch as is.
    We convert torch tensors to numpy arrays since np.pad doesn't work on tensors
    """
    imgs, labels = zip(*batch)                   # list of (np.ndarray, int)
    imgs = np.stack(imgs, axis=0)               # [B, 3, 224, 224] (np.float32)
    labels = np.asarray(labels, dtype=np.int64)  # [B]
    return imgs, labels


# implement using ImageNet
imagenet_val = ImageNet(
    root="data/ImageNet1k",
    split="val",
    transform=AlexNet_Weights.IMAGENET1K_V1.transforms()
)

# the dataloader automatically segregates the labels
val_dataloader = DataLoader(
    imagenet_val,
    batch_size=128,
    shuffle=False,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True,
    collate_fn=default_collate
)

In [86]:
for images, labels in val_dataloader:
    print(images.shape)
    print(labels.shape)
    break

/home/aleisley/Documents/mengai/ai231/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


(128, 3, 224, 224)
(128,)


#### Define the custom Conv2d

Here we just inherit from nn.Module so that it will work cleanly with pytorch even though the implementation of the computation is done by einops, einsum, and numpy.


In [115]:
class PatchMixin:
    def __init__(self, kernel_size: int, stride: int) -> None:
        super().__init__()
        self.kernel_size = kernel_size
        self.stride = stride

    def _patch_with_stride(self, x_pad: np.ndarray) -> np.ndarray:
        """
        Extracts k x k patches (kernel dims) from the input array with the given stride.

        Args:
            x (np.ndarray): Input array of shape (b, c, h, w).

        Returns:
            np.ndarray: Array of shape (b, c, h/2, w/2, k, k) containing the extracted patches.
        """
        windows = sliding_window_view(
            # type: ignore
            x_pad,
            window_shape=(self.kernel_size, self.kernel_size),
            axis=(-2, -1)  # type: ignore
        )

        # Stride by taking every second window in both height and width dimensions
        return windows[:, :, ::self.stride, ::self.stride, :, :]


class WeightsAndBiasMixin:
    def __init__(self, weights_loc: str, bias_loc: str, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.weights_loc = weights_loc
        self.bias_loc = bias_loc

    def init_weights_and_bias(self) -> None:
        print(f'Weights and biases: {weights_and_biases.keys()}')
        weights = np.load(self.weights_loc)
        bias = np.load(self.bias_loc)
        self.weight = nn.Parameter(torch.from_numpy(weights.astype('float32')))
        self.bias = torch.from_numpy(bias.astype('float32'))


class CustomConv2d(WeightsAndBiasMixin, PatchMixin, nn.Module):
    """
    2D Convolution layer using NumPy, Einops, and einsum.

    This isn't as flexible yet because it only supports very specific shapes.
    """

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size: int,
        stride: int = 1,
        padding: int = 0,
        weights_loc: str = '',
        bias_loc: str = '',
        # bias: bool = True
    ) -> None:
        super().__init__(weights_loc, bias_loc, kernel_size, stride)
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

        # just make sure to match features.0.weight and features.0.bias params from AlexNet
        # self.weight = nn.Parameter(torch.empty(
        #     out_channels, in_channels, kernel_size, kernel_size))

        # TODO: Change this
        test_kernel = np.arange(12).reshape(3, 1, 2, 2)
        self.weight = nn.Parameter(
            torch.from_numpy(test_kernel.astype('float32')))
        # self.bias = nn.Parameter(torch.empty(out_channels)) if bias else 0
        self.bias = np.arange(3)
        self.reset_parameters()

    def reset_parameters(self) -> None:
        """
        Since we are only going to be using this for inference, then we can just pass.
        We'll just fill this up when needed.
        """
        pass

    def _apply_padding(self, x: np.ndarray) -> np.ndarray:
        return np.pad(
            x,
            pad_width=((0, 0), (0, 0), (self.padding, self.padding),
                       (self.padding, self.padding)),
            mode='constant',
            constant_values=0
        )

    def forward(self, x: np.ndarray) -> np.ndarray:
        x_pad = self._apply_padding(x)
        patched_windows = self._patch_with_stride(x_pad)
        pre_activation = einsum(patched_windows, self.weight,
                                'b c w h kw kh, o c kw kh -> b o w h')

        return pre_activation + self.bias[None, :, None, None]  # type: ignore

In [108]:
class CustomReLU(nn.Module):
    """
    Custom ReLU activation function.
    """

    def forward(self, x: np.ndarray) -> np.ndarray:
        return np.maximum(x, 0.0)

In [109]:
class CustomMaxPool2d(PatchMixin, nn.Module):
    """
    Custom Max Pooling layer.
    """

    def __init__(self, kernel_size: int, stride: int) -> None:
        super().__init__(kernel_size, stride)

    def forward(self, x: np.ndarray) -> np.ndarray:
        patched_windows = self._patch_with_stride(x)
        return np.max(patched_windows, axis=(-2, -1))

In [ ]:
class CustomAdaptiveAvgPool2d(nn.Module):
    """
    Custom Adaptive Average Pooling layer (NumPy-based, inference only).

    Args:
        output_size (tuple[int, int]): desired (out_h, out_w)
    """

    def __init__(self, output_size: tuple[int, int]) -> None:
        super().__init__()
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            self.output_size = output_size

    def forward(self, x: np.ndarray) -> np.ndarray:
        """
        Args:
            x (np.ndarray): input of shape (b, c, h, w)
        Returns:
            np.ndarray: pooled output of shape (b, c, out_h, out_w)
        """
        b, c, h, w = x.shape
        out_h, out_w = self.output_size

        # compute region boundaries for adaptive pooling
        out = np.zeros((b, c, out_h, out_w), dtype=x.dtype)
        for i in range(out_h):
            h_start = int(np.floor(i * h / out_h))
            h_end = int(np.ceil((i+1) * h / out_h))
            for j in range(out_w):
                w_start = int(np.floor(j * w / out_w))
                w_end = int(np.ceil((j+1) * w / out_w))
                # (b, c, h_slice, w_slice)
                region = x[:, :, h_start:h_end, w_start:w_end]
                out[:, :, i, j] = region.mean(axis=(-2, -1))
        return out

In [116]:
x = np.arange(9).reshape(1, 1, 3, 3)
conv2d = CustomConv2d(1, 1, 2, stride=2, padding=1)
relu = CustomReLU()
maxpool = CustomMaxPool2d(kernel_size=2, stride=1)
with torch.no_grad():
    y = conv2d.forward(x)
    y = relu.forward(y)
    y = maxpool.forward(y)
    print(y)

[[[[ 43.]]

  [[140.]]

  [[237.]]]]
